

### Databricks Documentation RAG Ingestion Pipeline
### This notebook implements a high-quality RAG ingestion pipeline with:
### Data reading from Databricks tables
### Semantic chunking for context-aware splitting
### Metadata extraction for enhanced retrieval

#### Package Installation and Setup

In [0]:
# Install required packages
%pip install sentence-transformers scikit-learn nltk

# Download NLTK data
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

# Restart Python to use updated packages
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


[nltk_data] Downloading package punkt to /home/spark-
[nltk_data]     aef36c0d-db65-497b-829a-20/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/spark-
[nltk_data]     aef36c0d-db65-497b-829a-20/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
# STEP 1: Pre-download the model (run this ONCE first)
from ingestion_pipeline import download_model
download_model()  # Downloads ~90MB, takes about 30 seconds

This may take a minute on first run (~90MB download)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Model 'all-MiniLM-L6-v2' downloaded and cached successfully!
  Max sequence length: 256
  Embedding dimension: 384


True

In [0]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 165.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 MB 184.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 132.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 13.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from ingestion_pipeline import (
    read_databricks_docs,
    SemanticChunker,
    MetadataExtractor,
    process_document,
    process_all_documents,
    process_all_documents_v2
)

#### Step 1: Read Data from Databricks Table

In [0]:
# Read documentation data
docs_df = read_databricks_docs("databricks_databricks_documentation_dataset.v01.docs")
display(docs_df.limit(5))

Loaded 5169 documents from databricks_databricks_documentation_dataset.v01.docs


id url content 25277 https://docs.databricks.com/en/ingestion/bad-records.html Handle bad records and files 
Databricks provides a number of options for dealing with files that contain bad records. Examples of bad data include: 
Incomplete or corrupt records: Mainly observed in text based file formats like JSON and CSV. For example, a JSON record that doesn’t have a closing brace or a CSV record that doesn’t have as many columns as the header or first record of the CSV file. 
Mismatched data types: When the value for a column doesn’t have the specified or inferred data type. 
Bad field names: Can happen in all file formats, when the column name specified in the file or record has a different casing than the specified or inferred schema. 
Corrupted files: When a file cannot be read, which might be due to metadata or data corruption in binary file types such as Avro, Parquet, and ORC. On rare occasion, might be caused by long-lasting transient failures in the underlying storage system. 
Missing files: A file that was discovered during query analysis time and no longer exists at processing time. 
Use badRecordsPath
Use badRecordsPath
When you set badRecordsPath, the specified path records exceptions for bad records or files encountered during data loading. 
In addition to corrupt records and files, errors indicating deleted files, network connection exception, IO exception, and so on are ignored and recorded under the badRecordsPath. 
Note 
Using the badRecordsPath option in a file-based data source has a few important limitations: 
It is non-transactional and can lead to inconsistent results. 
Transient errors are treated as failures.

Unable to find input file
Unable to find input file
val df = spark.read .option("badRecordsPath", "/tmp/badRecordsPath") .format("parquet").load("/input/parquetFile") // Delete the input parquet file '/input/parquetFile' dbutils.fs.rm("/input/parquetFile") df.show() 
In the above example, since df.show() is unable to find the input file, Spark creates an exception file in JSON format to record the error. For example, /tmp/badRecordsPath/20170724T101153/bad_files/xyz is the path of the exception file. This file is under the specified badRecordsPath directory, /tmp/badRecordsPath. 20170724T101153 is the creation time of this DataFrameReader. bad_files is the exception type. xyz is a file that contains a JSON record, which has the path of the bad file and the exception/reason message.

Input file contains bad record
Input file contains bad record
// Creates a json file containing both parsable and corrupted records Seq("""{"a": 1, "b": 2}""", """{bad-record""").toDF().write.format("text").save("/tmp/input/jsonFile") val df = spark.read .option("badRecordsPath", "/tmp/badRecordsPath") .schema("a int, b int") .format("json") .load("/tmp/input/jsonFile") df.show() 
In this example, the DataFrame contains only the first parsable record ({"a": 1, "b": 2}). The second bad record ({bad-record) is recorded in the exception file, which is a JSON file located in /tmp/badRecordsPath/20170724T114715/bad_records/xyz. The exception file contains the bad record, the path of the file containing the record, and the exception/reason message. After you locate the exception files, you can use a JSON reader to process them. 25278 https://docs.databricks.com/en/ingestion/copy-into/configure-data-access.html Configure data access for ingestion 
This article describes how admin users can configure access to data in a bucket in Amazon S3 (S3) so that Databricks users can load data from S3 into a table in Databricks. 
This article describes the following ways to configure secure access to source data: 
(Recommended) Create a Unity Catalog volume. 
Create a Unity Catalog external location with a storage credential. 
Launch a compute resource that uses an AWS instance profile. 
Generate temporary credentials (an AWS access key ID, a secret key, and a session token). 
Before you begin
Before you begin
Before you configure access

#### Step 2: Test Semantic Chunking on a Sample Document

In [0]:
# Initialize semantic chunker
chunker = SemanticChunker(
    model_name='all-MiniLM-L6-v2',
    similarity_threshold=0.5,
    min_chunk_size=200,
    max_chunk_size=1000,
    overlap_sentences=2
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [0]:
# Test on a sample document
sample_doc = docs_df.first()
chunks = chunker.chunk_text(sample_doc['content'])


print(f"Created {len(chunks)} semantic chunks from sample document")
print("\nChunk Statistics:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1}:")
    print(f"  - Characters: {chunk.char_count}")
    print(f"  - Sentences: {chunk.sentence_count}")
    print(f"  - Preview: {chunk.text[:200]}...")

Created 11 semantic chunks from sample document

Chunk Statistics:

Chunk 1:
  - Characters: 403
  - Sentences: 3
  - Preview: Handle bad records and files  
Databricks provides a number of options for dealing with files that contain bad records. Examples of bad data include:  
Incomplete or corrupt records: Mainly observed i...

Chunk 2:
  - Characters: 799
  - Sentences: 6
  - Preview: Examples of bad data include:  
Incomplete or corrupt records: Mainly observed in text based file formats like JSON and CSV. For example, a JSON record that doesn’t have a closing brace or a CSV recor...

Chunk 3:
  - Characters: 361
  - Sentences: 3
  - Preview: Corrupted files: When a file cannot be read, which might be due to metadata or data corruption in binary file types such as Avro, Parquet, and ORC. On rare occasion, might be caused by long-lasting tr...


In [0]:
chunks

[SemanticChunk(text='Handle bad records and files  \nDatabricks provides a number of options for dealing with files that contain bad records. Examples of bad data include:  \nIncomplete or corrupt records: Mainly observed in text based file formats like JSON and CSV. For example, a JSON record that doesn’t have a closing brace or a CSV record that doesn’t have as many columns as the header or first record of the CSV file.', start_idx=0, end_idx=403, sentence_count=3, char_count=403),
 SemanticChunk(text='Examples of bad data include:  \nIncomplete or corrupt records: Mainly observed in text based file formats like JSON and CSV. For example, a JSON record that doesn’t have a closing brace or a CSV record that doesn’t have as many columns as the header or first record of the CSV file. Mismatched data types: When the value for a column doesn’t have the specified or inferred data type. Bad field names: Can happen in all file formats, when the column name specified in the file or record has

Step 3: Test Metadata Extraction

In [0]:
# Initialize metadata extractor
extractor = MetadataExtractor()

# Extract metadata from sample document
metadata = extractor.extract_metadata(
    doc_id=sample_doc['id'],
    url=sample_doc['url'],
    content=sample_doc['content']
)

print("Extracted Metadata:")
print(f"  Title: {metadata['title']}")
print(f"  Document Type: {metadata['document_type']}")
print(f"  URL Category: {metadata['url_category']}")
print(f"  URL Path: {metadata['url_path']}")
print(f"\nHeaders ({len(metadata['headers'])}):")
for header in metadata['headers'][:5]:
    print(f"  {'#' * header['level']} {header['text']}")

print(f"\nCode Blocks: {len(metadata['code_blocks'])}")
for i, cb in enumerate(metadata['code_blocks'][:3]):
    print(f"  Block {i+1}: {cb['language']} ({cb['length']} chars)")

print(f"\nLinks: {len(metadata['links'])}")
for link in metadata['links'][:5]:
    print(f"  - {link['text']}: {link['url']}")

print(f"\nKeywords:")
for key, values in metadata['keywords'].items():
    print(f"  {key}: {values[:5]}")

print(f"\nStatistics:")
for key, value in metadata['statistics'].items():
    print(f"  {key}: {value}")



Extracted Metadata:
  Title: None
  Document Type: general
  URL Category: ingestion
  URL Path: bad-records.html

Headers (0):

Code Blocks: 0

Links: 0

Keywords:
  sql_keywords: ['Delete']
  databricks_objects: ['schema']
  api_endpoints: []

Statistics:
  char_count: 3224
  word_count: 465
  line_count: 25
  header_count: 0
  code_block_count: 0
  link_count: 0


#### Step 4: Process Single Document with Both Functions

In [0]:
processed_chunks = process_document(
    doc_id=sample_doc['id'],
    url=sample_doc['url'],
    content=sample_doc['content'],
    chunker=chunker,
    extractor=extractor
)

print(f"Processed document into {len(processed_chunks)} chunks with metadata")
print(f"\nFirst chunk details:")
chunk = processed_chunks[0]
for key, value in chunk.items():
    if key != 'text':  # Skip full text for brevity
        print(f"  {key}: {value}")
print(f"\nText preview: {chunk['text'][:300]}...")

Processed document into 11 chunks with metadata

First chunk details:
  chunk_id: 25277_chunk_0
  doc_id: 25277
  url: https://docs.databricks.com/en/ingestion/bad-records.html
  chunk_index: 0
  total_chunks: 11
  char_count: 403
  sentence_count: 3
  start_idx: 0
  end_idx: 403
  doc_title: None
  doc_type: general
  url_category: ingestion
  url_path: bad-records.html
  has_code: False
  keywords: {'sql_keywords': ['Delete'], 'databricks_objects': ['schema'], 'api_endpoints': []}

Text preview: Handle bad records and files  
Databricks provides a number of options for dealing with files that contain bad records. Examples of bad data include:  
Incomplete or corrupt records: Mainly observed in text based file formats like JSON and CSV. For example, a JSON record that doesn’t have a closing ...


#### Step 5: Process All Documents (Batch Processing)

In [0]:
from ingestion_pipeline import process_all_documents

docs_df = spark.table("databricks_databricks_documentation_dataset.v01.docs")

chunks_df = process_all_documents(
    docs_df=docs_df,
    output_table="default.databricks_docs_processed_chunks",
    batch_size=100
)

Input: 5169 documents
Initializing models in driver...
Loading sentence-transformers model: all-MiniLM-L6-v2
✓ Model loaded successfully
✓ Models initialized successfully

Processing 5169 documents in batches of 100...

Batch 1/52: Processing 100 documents...
  Doc 22693: Processing 2872 chars...
  Doc 22693: Generated 11 chunks
  Doc 22694: Processing 4557 chars...
  Doc 22694: Generated 16 chunks
  Doc 22695: Processing 3147 chars...
  Doc 22695: Generated 12 chunks
  Doc 22696: Processing 3171 chars...
  Doc 22696: Generated 16 chunks
  Doc 22697: Processing 2485 chars...
  Doc 22697: Generated 11 chunks
  Doc 22698: Processing 2645 chars...
  Doc 22698: Generated 10 chunks
  Doc 22699: Processing 1816 chars...
  Doc 22699: Generated 8 chunks
  Doc 22700: Processing 2252 chars...
  Doc 22700: Generated 14 chunks
  Doc 22701: Processing 4250 chars...
  Doc 22701: Generated 16 chunks
  Doc 22702: Processing 3825 chars...
  Doc 22702: Generated 15 chunks
  Doc 22703: Processing 3556 ch

#### Step 6: Analyze Results

In [0]:
# Display processing statistics
print("Document Type Distribution:")
chunks_df.groupBy('doc_type').count().orderBy('count', ascending=False).show()


print("Chunk Size Statistics:")
chunks_df.select('char_count', 'sentence_count').describe().show()


# Display sample chunks
print("Sample Processed Chunks:")
display(chunks_df.limit(10))

Document Type Distribution:


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5650384099983296>, line 3
      1 # Display processing statistics
      2 print("Document Type Distribution:")
----> 3 chunks_df.groupBy('doc_type').count().orderBy('count', ascending=False).show()
      6 print("Chunk Size Statistics:")
      7 chunks_df.select('char_count', 'sentence_count').describe().show()

NameError: name 'chunks_df' is not defined

#### Testing scripts

In [0]:
"""
Simple test to verify chunking works without all the complexity.
Run this in your Databricks notebook to test if the basic chunking logic works.
"""

# Test 1: Can we create chunks with simple text splitting?
print("=" * 70)
print("TEST 1: Simple text splitting (no ML model)")
print("=" * 70)

def simple_chunk(text, chunk_size=500):
    """Simple chunking - split every N characters."""
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append({
            'chunk_id': f'chunk_{i}',
            'text': text[i:i+chunk_size],
            'char_count': len(text[i:i+chunk_size])
        })
    return chunks

# Get a sample document
sample = spark.table("databricks_databricks_documentation_dataset.v01.docs").limit(1).collect()[0]
print(f"Sample doc ID: {sample['id']}")
print(f"Content length: {len(sample['content'])} chars")

simple_chunks = simple_chunk(sample['content'])
print(f"✓ Simple chunking created {len(simple_chunks)} chunks")

# Test 2: Can we use the SemanticChunker?
print("\n" + "=" * 70)
print("TEST 2: Semantic chunking with sentence-transformers")
print("=" * 70)

try:
    from ingestion_pipeline import SemanticChunker
    
    print("Creating SemanticChunker...")
    chunker = SemanticChunker(
        similarity_threshold=0.5,
        min_chunk_size=200,
        max_chunk_size=1000,
        overlap_sentences=2
    )
    print("✓ SemanticChunker created")
    
    print(f"Chunking sample document...")
    semantic_chunks = chunker.chunk_text(sample['content'])
    print(f"✓ Semantic chunking created {len(semantic_chunks)} chunks")
    
    if len(semantic_chunks) > 0:
        print(f"\nFirst chunk preview:")
        print(f"  Text length: {semantic_chunks[0].char_count} chars")
        print(f"  Sentences: {semantic_chunks[0].sentence_count}")
        print(f"  Preview: {semantic_chunks[0].text[:150]}...")
    else:
        print("⚠️ WARNING: Semantic chunking returned 0 chunks!")
        print("This is the problem - let's debug why...")
        
        # Debug: Check sentence splitting
        sentences = chunker._split_into_sentences(sample['content'])
        print(f"\nDebug info:")
        print(f"  Total sentences: {len(sentences)}")
        if len(sentences) > 0:
            print(f"  First sentence: {sentences[0][:100]}...")
        
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 70)
print("TEST COMPLETE")
print("=" * 70)


TEST 1: Simple text splitting (no ML model)
Sample doc ID: 22693
Content length: 2872 chars
✓ Simple chunking created 6 chunks

TEST 2: Semantic chunking with sentence-transformers
Creating SemanticChunker...
Loading sentence-transformers model: all-MiniLM-L6-v2
✓ Model loaded successfully
✓ SemanticChunker created
Chunking sample document...
✓ Semantic chunking created 11 chunks

First chunk preview:
  Text length: 244 chars
  Sentences: 1
  Preview: Enable authentication to external Databricks services  
Databricks administrators can enable users to authenticate directly to external Databricks ser...

TEST COMPLETE


In [0]:
"""
Test the full process_document function to see if it works end-to-end.
"""

print("=" * 70)
print("TEST: Full process_document function")
print("=" * 70)

from ingestion_pipeline import SemanticChunker, MetadataExtractor, process_document

# Get sample document
sample = spark.table("databricks_databricks_documentation_dataset.v01.docs").limit(1).collect()[0]

print(f"Sample doc ID: {sample['id']}")
print(f"Content length: {len(sample['content'])} chars")

# Initialize chunker and extractor
print("\nInitializing models...")
chunker = SemanticChunker(
    similarity_threshold=0.5,
    min_chunk_size=200,
    max_chunk_size=1000,
    overlap_sentences=2
)
extractor = MetadataExtractor()
print("✓ Models initialized")

# Process the document
print("\nProcessing document...")
chunks = process_document(
    str(sample['id']),
    str(sample['url']),
    str(sample['content']),
    chunker,
    extractor
)

print(f"✓ Generated {len(chunks)} chunks")

if len(chunks) > 0:
    print(f"\nFirst chunk structure:")
    first_chunk = chunks[0]
    for key, value in first_chunk.items():
        if isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}... ({len(value)} chars)")
        else:
            print(f"  {key}: {value}")
    
    # Try to create a DataFrame from chunks
    print("\nTrying to create Spark DataFrame...")
    try:
        chunks_df = spark.createDataFrame(chunks)
        print(f"✓ DataFrame created with {chunks_df.count()} rows")
        print("\nDataFrame schema:")
        chunks_df.printSchema()
        print("\nSample row:")
        chunks_df.show(1, truncate=50, vertical=True)
    except Exception as e:
        print(f"✗ Error creating DataFrame: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ No chunks generated!")

print("\n" + "=" * 70)
print("TEST COMPLETE")
print("=" * 70)


TEST: Full process_document function
Sample doc ID: 22693
Content length: 2872 chars

Initializing models...
Loading sentence-transformers model: all-MiniLM-L6-v2
✓ Model loaded successfully
✓ Models initialized

Processing document...
✓ Generated 11 chunks

First chunk structure:
  chunk_id: 22693_chunk_0
  doc_id: 22693
  url: https://docs.databricks.com/en/admin/access-control/auth-external.html
  chunk_index: 0
  total_chunks: 11
  text: Enable authentication to external Databricks services  
Databricks administrators can enable users t... (243 chars)
  char_count: 244
  sentence_count: 1
  doc_title: 
  doc_type: general
  url_category: access-control
  url_path: auth-external.html
  has_code: false
  keywords: {"sql_keywords": ["from"], "databricks_objects": ["workspace"], "api_endpoints": []}

Trying to create Spark DataFrame...
✓ DataFrame created with 11 rows

DataFrame schema:
root
 |-- char_count: long (nullable = true)
 |-- chunk_id: string (nullable = true)
 |-- chunk_inde